In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.35)

from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.metrics import mean_squared_error, make_scorer, accuracy_score
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_val_predict
from sklearn.model_selection import KFold, StratifiedKFold, RepeatedKFold, RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import BaggingRegressor, BaggingClassifier, RandomForestRegressor, RandomForestClassifier

In [2]:
data_train = pd.read_csv('train_classification.csv')

In [3]:
# Response rate
data_train.host_response_rate = data_train.host_response_rate.str.replace('%','').astype(float)

# Host acceptance rate
data_train.host_acceptance_rate = data_train.host_acceptance_rate.str.replace('%','').astype(float)

In [4]:
# Imputing numeric

data_train = data_train.fillna(data_train.median())

C:\Users\julia\AppData\Local\Temp\ipykernel_332\2115613911.py:3: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data_train = data_train.fillna(data_train.median())


In [5]:
# apply everythingnto the test data
data_test = pd.read_csv('test_classification.csv')
data_test.host_response_rate = data_test.host_response_rate.str.replace('%','').astype(float)
data_test.host_acceptance_rate = data_test.host_acceptance_rate.str.replace('%','').astype(float)
data_test = data_test.fillna(data_test.median())

C:\Users\julia\AppData\Local\Temp\ipykernel_332\3534192112.py:5: FutureWarning: The default value of numeric_only in DataFrame.median is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  data_test = data_test.fillna(data_test.median())


In [6]:

data_train['host_is_superhost'] = data_train['host_is_superhost'].replace({'t': 1, 'f': 0})

data_train['host_response_time'] = data_train['host_response_time'].apply(lambda time: 1 if time == 'within an hour' or time == 'within a few hours' else 0)
data_test['host_response_time'] = data_test['host_response_time'].apply(lambda time: 1 if time == 'within an hour' or time == 'within a few hours' else 0)

data_train['room_type'] = data_train['room_type'].apply(lambda room: 1 if room in ['Private room', 'Entire home/apt', 'Hotel room'] else 0)
data_test['room_type'] = data_test['room_type'].apply(lambda room: 1 if room in ['Private room', 'Entire home/apt', 'Hotel room'] else 0)

data_train['property_type'] = data_train['property_type'].apply(lambda location: 1 if location in ['Shared room in home', 'Shared room in bungalow', 'Shared room in hostel', 'Shared room in rental unit', 'Shared room in condo'] else 0)
data_test['property_type'] = data_test['property_type'].apply(lambda location: 1 if location in ['Shared room in home', 'Shared room in bungalow', 'Shared room in hostel', 'Shared room in rental unit', 'Shared room in condo'] else 0)

data_train['host_neighbourhood'] = data_train['host_neighbourhood'].apply(lambda neighborhood: 1 if neighborhood in ['Lakeview', 'Mount Greenwood', 'Gold Coast', 'Chicago Loop', 'Edison Park'] else 0)
data_test['host_neighbourhood'] = data_test['host_neighbourhood'].apply(lambda neighborhood: 1 if neighborhood in ['Lakeview', 'Mount Greenwood', 'Gold Coast', 'Chicago Loop', 'Edison Park'] else 0)

data_train['host_total_listings_count'] = data_train['host_total_listings_count'].apply(lambda count: 1 if count <= 7 else 0)
data_test['host_total_listings_count'] = data_test['host_total_listings_count'].apply(lambda count: 1 if count <= 7 else 0)

data_train['bathrooms_text'] = data_train['bathrooms_text'].str.extract(r'(\d+)').astype(float)
data_test['bathrooms_text'] = data_test['bathrooms_text'].str.extract(r'(\d+)').astype(float)
data_train['bathrooms_text'] = data_train['bathrooms_text'].fillna(data_train['bathrooms_text'].median())
data_test['bathrooms_text'] = data_test['bathrooms_text'].fillna(data_test['bathrooms_text'].median())


In [7]:
# Convert float values to string
data_train['host_location'] = data_train['host_location'].astype(str)
data_test['host_location'] = data_test['host_location'].astype(str)

# List of cities to be considered as 'West'
west_cities = ['Chicago, IL', 'Lakeview', 'Mount Greenwood', 'Gold Coast', 'Chicago Loop', 'Edison Park']

# Map locations to 'West' (1), 'East' (0), and NaN (0)
data_train['host_location'] = data_train['host_location'].apply(lambda location: 1 if any(city in location for city in west_cities) else (0 if location == 'nan' else 0))
data_test['host_location'] = data_test['host_location'].apply(lambda location: 1 if any(city in location for city in west_cities) else (0 if location == 'nan' else 0))

data_train['host_has_profile_pic'] = data_train['host_has_profile_pic'].map({'t': 1, 'f': 0})
data_test['host_has_profile_pic'] = data_test['host_has_profile_pic'].map({'t': 1, 'f': 0})

data_train['host_identity_verified'] = data_train['host_identity_verified'].map({'t': 1, 'f': 0})
data_test['host_identity_verified'] = data_test['host_identity_verified'].map({'t': 1, 'f': 0})

data_train['has_availability'] = data_train['has_availability'].map({'t': 1, 'f': 0})
data_test['has_availability'] = data_test['has_availability'].map({'t': 1, 'f': 0})

data_train['instant_bookable'] = data_train['instant_bookable'].map({'t': 1, 'f': 0})
data_test['instant_bookable'] = data_test['instant_bookable'].map({'t': 1, 'f': 0})

data_train['host_verifications'] = data_train['host_verifications'].apply(lambda verifications: 1 if 'email' in verifications or 'phone' in verifications else 0)
data_test['host_verifications'] = data_test['host_verifications'].apply(lambda verifications: 1 if 'email' in verifications or 'phone' in verifications else 0)

neighborhoods_to_map_to_1 = ['Lake View', 'Lincoln Park', 'Near North Side', 'West Town', 'Logan Square']  # Add more neighborhoods as needed

data_train['neighbourhood_cleansed'] = data_train['neighbourhood_cleansed'].apply(lambda neighborhood: 1 if neighborhood in neighborhoods_to_map_to_1 else 0)
data_test['neighbourhood_cleansed'] = data_test['neighbourhood_cleansed'].apply(lambda neighborhood: 1 if neighborhood in neighborhoods_to_map_to_1 else 0)

columns_to_exclude = ['host_id', 'host_since', 'first_review', 'last_review']

data_train = data_train.drop(columns=columns_to_exclude, errors='ignore')
data_test = data_test.drop(columns=columns_to_exclude, errors='ignore')


In [8]:
data_train = data_train.fillna(data_train.median())
data_test = data_test.fillna(data_test.median())

In [9]:
X_train = data_train.drop(columns='host_is_superhost')
y_train = data_train['host_is_superhost']

X_test = data_test

In [10]:
cv_settings = RepeatedStratifiedKFold(n_splits=5,n_repeats=5,random_state=12)
model = RandomForestClassifier(random_state = 12, n_estimators = 100)

grid = {
    'max_samples': [0.25,0.5,0.75,0.9,1.0],
    'max_features':[4,5,6,7],
    'bootstrap':[True, False]
}
gscv = GridSearchCV(model, grid, cv=cv_settings, scoring = 'neg_root_mean_squared_error', n_jobs = -1)
gscv.fit(X_train, y_train)

C:\Users\julia\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
500 fits failed out of a total of 1000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
500 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\julia\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 732, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\julia\anaconda3\Lib\site-packages\sklearn\base.py", line 1151, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\julia\anaconda3\Lib\site-packages\sklearn\ensemble\_forest.py", line 402, in fit
    raise ValueEr

GridSearchCV(cv=RepeatedStratifiedKFold(n_repeats=5, n_splits=5, random_state=12),
             estimator=RandomForestClassifier(random_state=12), n_jobs=-1,
             param_grid={'bootstrap': [True, False],
                         'max_features': [4, 5, 6, 7],
                         'max_samples': [0.25, 0.5, 0.75, 0.9, 1.0]},
             scoring='neg_root_mean_squared_error')

In [11]:
gscv.best_params_

{'bootstrap': True, 'max_features': 6, 'max_samples': 0.9}

In [15]:
model = RandomForestClassifier(
    random_state = 12,
    n_estimators = 95,
    max_features = 6,
    max_samples = 0.9,
    bootstrap = True
)

In [16]:
model.fit(X_train, y_train)

RandomForestClassifier(max_features=6, max_samples=0.9, n_estimators=95,
                       random_state=12)

In [17]:
# Make predictions on the test data
preds = model.predict(X_test)

# Create a DataFrame with predictions and save it to a CSV file
output = pd.DataFrame({'id': data_test.id, 'predicted': preds})
output.to_csv('random_forest_classification_submission.csv', index=False)